In [ ]:
from itertools import product
import pandas as pd
import statsmodels.tsa.statespace.sarimax as sm
import numpy as np
import ray
from numba import jit
import os
from collections import defaultdict
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

In [ ]:
#@ray.remote
def fit_sarmiax(data, param, s):
    model=sm.SARIMAX(data, order=(param[0], param[1], param[2]),
                                            seasonal_order=(param[3], param[4], param[5], s)).fit(disp=-1)
    return model

In [ ]:
def optimizeSARIMA(data, parameters_list, s):
    """
        data - data to modelling
        parameters_list - list with (p, q, P, Q) tuples
        d - integration order in ARIMA model
        D - seasonal integration order
        s - length of season
    """
   
    results = []
    params = []
    i = 1
    for param in parameters_list:
        print('Tested {} from {}'.format(i, len(parameter_list)))
        i+=1
        try:
            model= fit_sarmiax(data, param, s)
            results.append(model.aic)
            params.append(param)
        except Exception as err:
            continue
    #results = ray.get(results)
    #aic = [model.aic for model in results]
    d = {'aic': results, 'paramters':parameters_list}
    result_table = pd.DataFrame(d)
    # sort, lower AIC crit -> better
    result_table = result_table.sort_values(by='aic', ascending=True).reset_index(drop=True)
    return result_table

In [ ]:
def fitSARIMAX(datas, column, installations, parametes, s):
    #results = []
    arparams = []
    maparams = []
    seasonalarparams = []
    seasonalmaparams = []
    i = 1
    for installation in installations:
        print('Fitted {} from {}'.format(i, len(installations)))
        tmp = datas[installation]
        model = fit_sarmiax(tmp[column], parameters, s)
        arparams.append(model.arparams)
        maparams.append(model.maparams)
        seasonalarparams.append(0)
        #seasonalarparams.append(model.seasonalarparams)
        seasonalmaparams.append(model.seasonalmaparams)
        #results.append(model)
        i+=1
    #results = ray.get(results)
    #d = {'installations': installations, 'model': results}
    return arparams, maparams, seasonalarparams, seasonalmaparams

In [ ]:
def prepare_data(path):
    data = pd.read_csv(os.path.join(path, 'airly_data.csv'))
    data.start_date = pd.to_datetime(data.start_date)
    data = data.sort_values(by='start_date').reset_index(drop=True)
    return data

In [ ]:
def fit_data(installation, df):
    data = df.copy()
    if installation == 'global':
        data = data.groupby(data.start_date).mean().drop(['installation_id'], 1).sort_index().reset_index(drop=True)
        return data
    else:
        data = data[data.installation_id == installation].set_index('start_date').sort_index().reset_index(drop=True)
        return data

In [ ]:
def installations_data(data, installations):
    datas = defaultdict()
    for installation in installations:
        datas[installation] = fit_data(installation, data)
    return datas

In [ ]:
def installations(data):
    return data.installation_id.drop_duplicates().tolist()

In [ ]:
#ray.init()

In [ ]:
ps = range(1, 4)
ds = range(0, 2)
qs = range(0, 3)
Ps = range(0, 3)
Ds = range(0, 2)
Qs = range(0, 3)
s =  24
# ps = range(1, 2)
# ds = range(0, 1)
# qs = range(1, 2)
# Ps = range(1, 2)
# Ds = range(0, 1)
# Qs = range(1, 2)
# s =  24
parameter_list = list(product(ps, ds, qs, Ps, Ds, Qs))
len(parameter_list)

In [ ]:
data = prepare_data('/home/grzegorz/Academia/lectures/Longitudinal_and_Functional_Data_Analysis')

In [ ]:
df = fit_data('global', data)

In [ ]:
%time result_table = optimizeSARIMA(df.air_quality_index_value, parameter_list, s)

In [ ]:
parameters = result_table.iloc[0, 1]
parameters

In [ ]:
installations_ids = installations(data)
#installations_ids = installations_ids[:200]

In [ ]:
datas = installations_data(data, installations_ids)

In [ ]:
%time arparams, maparams, seasonalarparams, seasonalmaparams = fitSARIMAX(datas, 'PM10', installations_ids, parameters, s)

In [ ]:
#ray.shutdown()

In [ ]:
# arparams = [models[models.installations == installation].iloc[0, 1].arparams for installation in installations_ids]
# maparams = [models[models.installations == installation].iloc[0, 1].maparams for installation in installations_ids]
# seasonalarparams = [models[models.installations == installation].iloc[0, 1].seasonalarparams for installation in installations_ids]
# seasonalmaparams = [models[models.installations == installation].iloc[0, 1].seasonalmaparams for installation in installations_ids]

In [ ]:
coeffs = defaultdict()
coeffs['installation'] = installations_ids
for i in range(parameters[0]):
    coeffs['arparams_' + str(i)] = [arparams[j][i] for j in range(len(arparams))]
for i in range(parameters[2]):
    coeffs['maparams_' + str(i)] = [maparams[j][i] for j in range(len(maparams))]
for i in range(parameters[3]):
    coeffs['seasonalarparams_' + str(i)] = [seasonalarparams[j][i] for j in range(len(seasonalarparams))]
for i in range(parameters[5]):
    coeffs['seasonalmaparams_' + str(i)] = [seasonalmaparams[j][i] for j in range(len(seasonalmaparams))]
coeffs = pd.DataFrame(coeffs)
coeffs.to_csv('/home/grzegorz/Academia/lectures/Longitudinal_and_Functional_Data_Analysis/models.csv')

In [ ]:
scaler = StandardScaler()

In [ ]:
coeff_np = scaler.fit_transform(np.array(coeffs.iloc[:, 1:]))
epss = [2**i for i in range(-5, 5)]

In [ ]:
clustering = []
for eps in epss:
    clustering.append(DBSCAN(eps=eps, min_samples=5, algorithm='brute', n_jobs=-1).fit_predict(coeff_np))